去年共有278天领用了物料，记录在278张Excel表中。现在，老板想将所有物料按领用量从高到低排序，以便查看前10种领用最多的物料的情况。如果手工操作，需要把每张工作表的内容合并在一起，再用数据透视表来做。而对于这种重复操作，Phthon最擅长。我们可以用实例22 （https://zhuanlan.zhihu.com/p/140056868） 的方法用Python将所有工作表合在一起，再用数据透视表操作。但Python还有个功能强大的`pandas`(Python Data Analysis Library)库，专门用于做数据分析。它包含很多数据处理的函数和方法，可帮助我们快捷高效地处理数据。现在，我们就来演示如何用`pandas`统计一个Excel工作簿中278张表的数据并汇总，排序。我们先导入`pandas`库，为方便后续简化书写，大家都习惯于给它起个小名叫`pd`。

In [1]:
import pandas as pd

然后我们先用`pd.read_excel()`打开第一张工作表，试试水，打开后存入变量`df`。传入要打开的工作簿，即`'日领料单.xlsx'`。数据的字段名在第三行，指定`header=2`。因为header是用0表示第一行，所以第三行对应的索引为2。第一张表的名称叫`01-03`，所以指定参数`sheet_name = '01-03'`。打开后，用`df.head()`看一下效果，这个函数值看头几行数据，括号内不填具体数量，则默认头五行。相对应的，`df.tail()`则是看末尾5行。
![](images\header.png)

In [61]:
df = pd.read_excel('日领料单.xlsx' ,header=2, sheet_name = '01-03')
df.head()

,域,挑库单号,制造组,需求数量,工作订单编号,工作中心,状态,库位,单位,项次,物料编号,架位,批号,批号批数量,产品型号,物料描述
0,AAA,377355.0,NaN,3.0,A19X9239156,QQ31,Complete,WH-B,EA,1.0,11010XR0073700,LC000001,PC00001,3.0,A00001,面壳
1,AAA,377355.0,NaN,10.0,A19X9239156,QQ31,Complete,WH-B,EA,2.0,11020XR0025500,LC000002,PC00002,10.0,A00002,底壳
2,AAA,377355.0,NaN,3.0,A19X9239156,QQ31,Complete,WH-B,EA,3.0,11030XR0013200,LC000003,PC00003,3.0,A00003,电池门
3,AAA,377355.0,NaN,3.0,A19X9239156,QQ31,Complete,WH-B,EA,4.0,11140XR0000100,LC000004,PC00004,3.0,A00004,底壳/后盖
4,AAA,377355.0,NaN,3.0,A19X9239156,QQ31,Complete,WH,EA,5.0,12121XR0172300,LC000005,PC00005,3.0,A00005,贴纸


数据显示与Excel表中完全一致，那就可以开始下一步了，即按照“物料编号”和“物料描述”字段将“批号批数量”加总。这里将使用到`groupby()`，它的作用是分组聚合，有点类似数据透视表中的“行”。此处我们按“物料编号”和“物料描述”分组聚合数据，并按“批号批数量”加总`['批号批数量'].sum()`。因为汇总数据后，行会减少(从191行减少到163行)，所以需要重设行编号`reset_index()`，按0~162重新编号。

In [66]:
#按物料编号加总领料数量
df_sum = df.groupby(['物料编号','物料描述'])['批号批数量'].sum().reset_index()
df_sum.tail() #看尾部5行的数据

,物料编号,物料描述,批号批数量
158,73009XR0000290,集成电路,110.0
159,73099XR0000580,集成电路,400.0
160,75008XR0000035,开关,120.0
161,75008XR0000043,开关,1200.0
162,77020XR0000013,话筒(麦克风),110.0


然后我们用`sort_values()`排序，排序规则是从大到小`ascending = False`，并看前10项的数据`head(10)`。这个跟Excel中的数据透视表得到的结果完全一致。
![](images\pivot_table.png)

In [68]:
df_sum.sort_values('批号批数量',ascending = False).head(10)

,物料编号,物料描述,批号批数量
111,62010XR050150A,电容,1900.0
118,62011XR0501046,电容,1210.0
161,75008XR0000043,开关,1200.0
69,61010XR0000000,电阻,1000.0
96,62010XR0101044,电容,830.0
52,46000XR0420060,螺丝,570.0
130,63119XR0000160,二极管,540.0
122,62018XR0500101,电容,540.0
155,71311XR1320002,振荡器,490.0
81,61011XR1000000,电阻,450.0


以上，是对单个工作表的处理，下面我们用同样的方式遍历全部278张工作表，然后汇总数据。先新建一个空的数据框`result`，用于存储汇总所有工作表的结果。然后通过传入参数`sheet_name = None`（即不指定工作表，则全部读取），读取整个Excel文件中的所有工作表。然后按工作表名遍历所有工作表，分组聚合，加总“批号批数量”，将汇总后的数据框`df_sum`通过连接函数`concat`增加到`result`中。在此处，`concat`有点类似于列表中的`append`增加元素的功能，这里增加的是DataFrame。其作用的示意图如下。
![](images\concat.png)

In [74]:
#新建一个DataFrame用于存储汇总所有工作表的结果
result = pd.DataFrame()

#读取整个Excel文件中的所有表
df = pd.read_excel('日领料单.xlsx', header=2, sheet_name = None)
#按表名遍历，处理数据
for sheet_name in df.keys():    
    #按物料编号加总领料数量
    df_sum = df[sheet_name].groupby(['物料编号','物料描述'])['批号批数量'].sum().reset_index()    

    result = pd.concat([result,df_sum])
    
result.head()

,物料编号,物料描述,批号批数量
0,10010XR0239300,面壳,110.0
1,10010XR0244500,面壳,120.0
2,10011XR0214500,面壳,30.0
3,10020XR0157200,底壳,30.0
4,10020XR0174800,底壳,110.0


以上，`df.keys()`存有所有工作表名，共278个。通过`for`循环逐个从其中取出名字，然后通过名字读取数据及汇总。

In [78]:
df.keys()

odict_keys(['01-03', '01-04', '01-05', '01-07', '01-08', '01-09', '01-10', '01-11', '01-12', '01-14', '01-15', '01-16', '01-17', '01-18', '01-19', '01-21', '01-22', '01-23', '01-24', '01-25', '01-26', '01-28', '01-29', '01-30', '02-12', '02-13', '02-14', '02-15', '02-16', '02-18', '02-19', '02-20', '02-21', '02-22', '02-23', '02-25', '02-26', '02-27', '02-28', '03-04', '03-05', '03-06', '03-07', '03-10', '03-11', '03-12', '03-13', '03-14', '03-15', '03-16', '03-18', '03-19', '03-20', '03-21', '03-22', '03-23', '03-24', '03-25', '03-26', '03-27', '03-28', '03-29', '03-30', '04-02', '04-03', '04-04', '04-06', '04-09', '04-10', '04-11', '04-12', '04-13', '04-14', '04-15', '04-16', '04-17', '04-18', '04-19', '04-20', '04-21', '04-22', '04-23', '04-24', '04-25', '04-26', '04-27', '04-29', '04-30', '05-03', '05-04', '05-06', '05-07', '05-08', '05-09', '05-10', '05-11', '05-13', '05-14', '05-15', '05-16', '05-17', '05-20', '05-21', '05-22', '05-23', '05-24', '05-25', '05-27', '05-28', '05-29'

数据汇总在一起后，需要再最后分类汇总一下（因为每天领取的物料是有重复的，需要将相同物料编号对应的数量加总），并降序排序。最终得到领用量最多的10个物料。

In [72]:
final = result.groupby(['物料编号','物料描述'])['批号批数量'].sum().reset_index().sort_values('批号批数量',ascending = False)
final.head(10)

,物料编号,物料描述,批号批数量
2798,61010XR0000000,电阻,116335.0
3793,63563XR0000030,二极管,84377.0
2871,61010XR1100300,电阻,78209.0
2802,61010XR0010300,电阻,74348.0
1656,46000XR0420060,螺丝,57609.0
3203,62010XR0101056,电容,56875.0
2901,61010XR1220202,电阻,51675.0
2818,61010XR0047000,电阻,51336.0
3224,62010XR0161048,电容,50766.0
3023,61011XR1000000,电阻,47425.0


我们还可以将最终结果保存为Excel文件，以便在Excel中操作。

In [59]:
final.to_excel("汇总.xlsx")